In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import sklearn 
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.svm import SVC,SVR
from sklearn.linear_model import SGDRegressor,SGDClassifier,RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,NearestNeighbors
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from catboost import CatBoost,CatBoostClassifier,CatBoostRegressor
from sklearn.preprocessing import OneHotEncoder,StandardScaler,MinMaxScaler,RobustScaler,MaxAbsScaler
from sklearn.compose import make_column_transformer

In [2]:
!ls

00-main.ipynb	  cleaned-train-data.csv  model-CatBoost-0.sav	test.csv
1-submission.csv  data.zip		  old_main.ipynb	train.csv
catboost_info	  model-CatBoost-0.pkl	  sub-0.csv


In [3]:
data = pd.read_csv('./cleaned-train-data.csv')

In [4]:
sex_converter = {'male':0,'female':1}
embarked_converter = {'S':0,'C':1,'Q':2}

In [5]:
new_embarked_col = []
for a in data['Embarked']:
    new_embarked_col.append(embarked_converter[a])
new_sex_col = []
for a in data['Sex']:
    new_sex_col.append(sex_converter[a])

In [6]:
data['Embarked'] = new_embarked_col
data['Sex'] = new_sex_col

In [7]:
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex              int64
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked         int64
dtype: object

In [8]:
test_data = pd.read_csv('./test.csv')

## preproccessing

In [9]:
X = data.drop('Survived',axis=1)
y = data['Survived']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [10]:
X.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex              int64
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked         int64
dtype: object

## .toarray() vs compact

### compact

In [11]:
# # 0.767
# avg = []
# for _ in range(10):
#   ct = make_column_transformer(
#       (OneHotEncoder(handle_unknown='ignore'),['Name','Sex','Ticket','Cabin','Embarked']),
#       (StandardScaler(),['PassengerId','Pclass','Age','SibSp','Parch','Fare'])
#   )
#   ct.fit(X)
#   X_train_pre = ct.transform(X_train)
#   X_train_pre = ct.transform(X_train)
#   X_test_pre = ct.transform(X_test)
#   model = RandomForestClassifier()
#   model.fit(X_train_pre,y_train)
#   print(f'model score : {model.score(X_test_pre,y_test)}')
#   avg.append(model.score(X_test_pre,y_test))
# print(np.mean(avg))

### .toarrray()

In [12]:
# # 0.76589
# avg = []
# for _ in range(10):
#   ct = make_column_transformer(
#       (OneHotEncoder(handle_unknown='ignore'),['Name','Sex','Ticket','Cabin','Embarked']),
#       (StandardScaler(),['PassengerId','Pclass','Age','SibSp','Parch','Fare'])
#   )
#   ct.fit(X)
#   X_train_pre = ct.transform(X_train)
#   X_train_pre = ct.transform(X_train)
#   X_test_pre = ct.transform(X_test)
#   model = RandomForestClassifier()
#   model.fit(X_train_pre,y_train)
#   print(f'model score : {model.score(X_test_pre,y_test)}')
#   avg.append(model.score(X_test_pre,y_test))
# print(np.mean(avg))

## best preproccessing way

In [13]:
# preproccesings = [StandardScaler(),RobustScaler(),MinMaxScaler(),MaxAbsScaler()]
# for preprocces in preproccesings:
#   ct = make_column_transformer(
#       (OneHotEncoder(handle_unknown='ignore'),['Name','Sex','Ticket','Cabin','Embarked']),
#       (preprocces,['PassengerId','Pclass','Age','SibSp','Parch','Fare'])
#   )
#   ct.fit(X)
#   X_train_pre = ct.transform(X_train)
#   X_test_pre = ct.transform(X_test)
#   model = RandomForestClassifier()
#   model.fit(X_train_pre,y_train)
#   print(f'{preprocces} : {model.score(X_test_pre,y_test)}')

## RobustScaler

In [14]:
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex              int64
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked         int64
dtype: object

In [15]:
# 0.76589
ct = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore'),['Name','Ticket','Cabin']),
    (RobustScaler(),['PassengerId','Pclass','Age','Sex','SibSp','Parch','Fare','Embarked'])
)
ct.fit(X)
X_train_pre = ct.transform(X_train)
X_train_pre = ct.transform(X_train)
X_test_pre = ct.transform(X_test)

In [16]:
X_train = X_train_pre
X_test = X_test_pre

In [17]:
# from sklearn.svm import SVC,SVR
# from sklearn.linear_model import SGDRegressor,SGDClassifier,RidgeClassifier,LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,NearestNeighbors
# from sklearn.naive_bayes import GaussianNB
# from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
# from catboost import CatBoost,CatBoostClassifier,CatBoostRegressor
# models = [
#     ['SVC',SVC()],
#     ['SGDClassifier',SGDClassifier()],
#     ['RidgeClassifier',RidgeClassifier()],
#     ['KNeighborsClassifier',KNeighborsClassifier()],
#     ['RandomForestClassifier',RandomForestClassifier()],
#     ['CatBoost',CatBoost()],
#     ['CatBoostClassifier',CatBoostClassifier()],
#     ['RandomForestClassifier2',RandomForestClassifier(n_estimators=100, oob_score = True, random_state = 1)],
#     ['LogisticRegression',LogisticRegression(solver='liblinear')],
# ]
# maxes = []
# for _ in range(2):
#   results = {}
#   for model in models:
#       print(model)
#       model_name = model[0]
#       model = model[1]
#       model.fit(X_train,y_train)
#       try:
#           accuracy = model.score(X_test,y_test)
#           results[model_name] = accuracy
#       except:
#           preds = model.predict(X_test)
#           preds = np.round(preds)
#           accuracy = accuracy_score(y_test,preds)
#           results[model_name] = accuracy
#       print(f'{model_name} : {accuracy}')
#   print(results)
#   maxes.append(max(results, key=results.get))

In [18]:
maxes

NameError: name 'maxes' is not defined

In [19]:
from collections import Counter
occurence_count = Counter(maxes)
print(occurence_count.most_common(1)[0][0])

NameError: name 'maxes' is not defined

In [20]:
results

NameError: name 'results' is not defined

In [21]:
# import pickle
# model = pickle.load(open('./model-CatBoost-0.sav','rb'))

In [22]:
# test_data.head()

In [23]:
# test_data.isna().sum()

In [24]:
# data.dtypes

In [25]:
# import random
# test_data['Ticket'] = test_data['Ticket'].fillna(random.choice(test_data['Ticket']))
# test_data['Age'] = test_data['Age'].fillna(test_data['Age'].mean())
# test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].mean())
# test_data['Cabin'] = test_data['Cabin'].fillna(random.choice(test_data['Cabin']))
# test_data['Embarked'] = test_data['Embarked'].fillna(random.choice(test_data['Embarked']))
# test_data['Cabin'] = test_data['Cabin'].fillna(random.choice(test_data['Cabin']))

In [26]:
# test_data.isna().sum()

In [27]:
# test_data['Cabin']

In [28]:
# preds = model.predict(ct.transform(test_data))
# preds = np.round(preds)

In [29]:
# submission = pd.DataFrame({'PassengerId':test_data['PassengerId'],'Survived':preds})

In [30]:
# submission = submission.astype(int)

In [31]:
# submission.dtypes

In [32]:
# submission.to_csv('./1-submission.csv',index=False)

## 2nd try

In [33]:
## grid = {'learning_rate': [0.1,1.0,0.01,0.001],
##         'depth': [round(2.5),5],
##         'l2_leaf_reg': [round(7.5),10]}
# grid = {'learning_rate': [0.1,1.0,0.01,0.001],
#         'l2_leaf_reg': [round(2.5),5,round(7.5),10],
#         'depth': [round(7.5),10,5,round(2.5)]}
model = CatBoostClassifier()
model.grid_search(grid,X_train,y_train)
preds = model.predict(X_test)
print(accuracy_score(y_test,np.round(preds)))
import pickle
pickle.dump(model,open('./model-CatBoostClassifier-0.sav','wb'))
pickle.dump(model,open('./model-CatBoostClassifier-0.pkl','wb'))

0:	learn: 0.6572543	test: 0.6573085	best: 0.6573085 (0)	total: 137ms	remaining: 2m 16s
1:	learn: 0.6288478	test: 0.6289189	best: 0.6289189 (1)	total: 206ms	remaining: 1m 42s
2:	learn: 0.6045778	test: 0.6046220	best: 0.6046220 (2)	total: 276ms	remaining: 1m 31s
3:	learn: 0.5855399	test: 0.5855585	best: 0.5855585 (3)	total: 350ms	remaining: 1m 27s
4:	learn: 0.5707572	test: 0.5707467	best: 0.5707467 (4)	total: 428ms	remaining: 1m 25s
5:	learn: 0.5590693	test: 0.5591420	best: 0.5591420 (5)	total: 503ms	remaining: 1m 23s
6:	learn: 0.5494752	test: 0.5494954	best: 0.5494954 (6)	total: 582ms	remaining: 1m 22s
7:	learn: 0.5410163	test: 0.5409934	best: 0.5409934 (7)	total: 657ms	remaining: 1m 21s
8:	learn: 0.5347316	test: 0.5346518	best: 0.5346518 (8)	total: 729ms	remaining: 1m 20s
9:	learn: 0.5297353	test: 0.5295986	best: 0.5295986 (9)	total: 806ms	remaining: 1m 19s
10:	learn: 0.5249124	test: 0.5247333	best: 0.5247333 (10)	total: 886ms	remaining: 1m 19s
11:	learn: 0.5213876	test: 0.5212276	best

93:	learn: 0.4854482	test: 0.4846852	best: 0.4846852 (93)	total: 7.23s	remaining: 1m 9s
94:	learn: 0.4854235	test: 0.4846588	best: 0.4846588 (94)	total: 7.3s	remaining: 1m 9s
95:	learn: 0.4854171	test: 0.4846588	best: 0.4846588 (95)	total: 7.37s	remaining: 1m 9s
96:	learn: 0.4852853	test: 0.4845072	best: 0.4845072 (96)	total: 7.45s	remaining: 1m 9s
97:	learn: 0.4852126	test: 0.4844230	best: 0.4844230 (97)	total: 7.52s	remaining: 1m 9s
98:	learn: 0.4850911	test: 0.4842360	best: 0.4842360 (98)	total: 7.59s	remaining: 1m 9s
99:	learn: 0.4850501	test: 0.4841966	best: 0.4841966 (99)	total: 7.67s	remaining: 1m 9s
100:	learn: 0.4849935	test: 0.4841198	best: 0.4841198 (100)	total: 7.75s	remaining: 1m 8s
101:	learn: 0.4849261	test: 0.4840646	best: 0.4840646 (101)	total: 7.83s	remaining: 1m 8s
102:	learn: 0.4848609	test: 0.4839775	best: 0.4839775 (102)	total: 7.91s	remaining: 1m 8s
103:	learn: 0.4847925	test: 0.4839149	best: 0.4839149 (103)	total: 7.98s	remaining: 1m 8s
104:	learn: 0.4847499	tes

186:	learn: 0.4811878	test: 0.4809438	best: 0.4809438 (186)	total: 14.4s	remaining: 1m 2s
187:	learn: 0.4811788	test: 0.4809400	best: 0.4809400 (187)	total: 14.4s	remaining: 1m 2s
188:	learn: 0.4811698	test: 0.4809400	best: 0.4809400 (187)	total: 14.5s	remaining: 1m 2s
189:	learn: 0.4811618	test: 0.4809400	best: 0.4809400 (187)	total: 14.6s	remaining: 1m 2s
190:	learn: 0.4811544	test: 0.4809400	best: 0.4809400 (187)	total: 14.7s	remaining: 1m 2s
191:	learn: 0.4811468	test: 0.4809400	best: 0.4809400 (187)	total: 14.8s	remaining: 1m 2s
192:	learn: 0.4811259	test: 0.4809226	best: 0.4809226 (192)	total: 14.8s	remaining: 1m 2s
193:	learn: 0.4810903	test: 0.4808758	best: 0.4808758 (193)	total: 14.9s	remaining: 1m 1s
194:	learn: 0.4810805	test: 0.4808701	best: 0.4808701 (194)	total: 15s	remaining: 1m 1s
195:	learn: 0.4810721	test: 0.4808719	best: 0.4808701 (194)	total: 15.1s	remaining: 1m 1s
196:	learn: 0.4810645	test: 0.4808715	best: 0.4808701 (194)	total: 15.1s	remaining: 1m 1s
197:	learn: 

279:	learn: 0.4798659	test: 0.4802219	best: 0.4802219 (279)	total: 21.6s	remaining: 55.6s
280:	learn: 0.4798596	test: 0.4802219	best: 0.4802219 (280)	total: 21.7s	remaining: 55.6s
281:	learn: 0.4798522	test: 0.4802219	best: 0.4802219 (281)	total: 21.8s	remaining: 55.5s
282:	learn: 0.4798448	test: 0.4802219	best: 0.4802219 (282)	total: 21.9s	remaining: 55.4s
283:	learn: 0.4798192	test: 0.4802170	best: 0.4802170 (283)	total: 22s	remaining: 55.3s
284:	learn: 0.4798119	test: 0.4802170	best: 0.4802170 (283)	total: 22s	remaining: 55.3s
285:	learn: 0.4797948	test: 0.4802130	best: 0.4802130 (285)	total: 22.1s	remaining: 55.2s
286:	learn: 0.4797887	test: 0.4802151	best: 0.4802130 (285)	total: 22.2s	remaining: 55.1s
287:	learn: 0.4797822	test: 0.4802127	best: 0.4802127 (287)	total: 22.3s	remaining: 55.1s
288:	learn: 0.4797727	test: 0.4802127	best: 0.4802127 (288)	total: 22.4s	remaining: 55s
289:	learn: 0.4797539	test: 0.4801916	best: 0.4801916 (289)	total: 22.4s	remaining: 54.9s
290:	learn: 0.47

372:	learn: 0.4789164	test: 0.4799226	best: 0.4799188 (361)	total: 29s	remaining: 48.7s
373:	learn: 0.4789090	test: 0.4799226	best: 0.4799188 (361)	total: 29.1s	remaining: 48.7s
374:	learn: 0.4789031	test: 0.4799226	best: 0.4799188 (361)	total: 29.2s	remaining: 48.6s
375:	learn: 0.4788962	test: 0.4799226	best: 0.4799188 (361)	total: 29.2s	remaining: 48.5s
376:	learn: 0.4788917	test: 0.4799226	best: 0.4799188 (361)	total: 29.3s	remaining: 48.4s
377:	learn: 0.4788851	test: 0.4799226	best: 0.4799188 (361)	total: 29.4s	remaining: 48.4s
378:	learn: 0.4788780	test: 0.4799226	best: 0.4799188 (361)	total: 29.5s	remaining: 48.3s
379:	learn: 0.4788717	test: 0.4799226	best: 0.4799188 (361)	total: 29.6s	remaining: 48.2s
380:	learn: 0.4788651	test: 0.4799226	best: 0.4799188 (361)	total: 29.6s	remaining: 48.1s
381:	learn: 0.4788584	test: 0.4799150	best: 0.4799150 (381)	total: 29.7s	remaining: 48.1s
382:	learn: 0.4788519	test: 0.4799150	best: 0.4799150 (381)	total: 29.8s	remaining: 48s
383:	learn: 0.

...

581:	learn: 0.4664489	test: 0.4777558	best: 0.4777488 (570)	total: 9m 18s	remaining: 6m 41s
582:	learn: 0.4664381	test: 0.4777553	best: 0.4777488 (570)	total: 9m 19s	remaining: 6m 40s
583:	learn: 0.4664287	test: 0.4777553	best: 0.4777488 (570)	total: 9m 20s	remaining: 6m 39s
584:	learn: 0.4664173	test: 0.4777556	best: 0.4777488 (570)	total: 9m 21s	remaining: 6m 38s
585:	learn: 0.4664079	test: 0.4777557	best: 0.4777488 (570)	total: 9m 22s	remaining: 6m 37s
586:	learn: 0.4663971	test: 0.4777550	best: 0.4777488 (570)	total: 9m 23s	remaining: 6m 36s
587:	learn: 0.4663877	test: 0.4777546	best: 0.4777488 (570)	total: 9m 24s	remaining: 6m 35s
588:	learn: 0.4663783	test: 0.4777550	best: 0.4777488 (570)	total: 9m 25s	remaining: 6m 34s
589:	learn: 0.4663555	test: 0.4777452	best: 0.4777452 (589)	total: 9m 26s	remaining: 6m 33s
590:	learn: 0.4663444	test: 0.4777449	best: 0.4777449 (590)	total: 9m 27s	remaining: 6m 32s
591:	learn: 0.4663351	test: 0.4777462	best: 0.4777449 (590)	total: 9m 27s	remain

670:	learn: 0.4653193	test: 0.4777133	best: 0.4777122 (659)	total: 10m 42s	remaining: 5m 15s
671:	learn: 0.4653067	test: 0.4777139	best: 0.4777122 (659)	total: 10m 43s	remaining: 5m 14s
672:	learn: 0.4652975	test: 0.4777148	best: 0.4777122 (659)	total: 10m 44s	remaining: 5m 13s
673:	learn: 0.4652859	test: 0.4777134	best: 0.4777122 (659)	total: 10m 45s	remaining: 5m 12s
674:	learn: 0.4652751	test: 0.4777138	best: 0.4777122 (659)	total: 10m 46s	remaining: 5m 11s
675:	learn: 0.4652650	test: 0.4777139	best: 0.4777122 (659)	total: 10m 47s	remaining: 5m 10s
676:	learn: 0.4652204	test: 0.4776882	best: 0.4776882 (676)	total: 10m 48s	remaining: 5m 9s
677:	learn: 0.4652113	test: 0.4776887	best: 0.4776882 (676)	total: 10m 49s	remaining: 5m 8s
678:	learn: 0.4652021	test: 0.4776887	best: 0.4776882 (676)	total: 10m 50s	remaining: 5m 7s
679:	learn: 0.4651915	test: 0.4776897	best: 0.4776882 (676)	total: 10m 51s	remaining: 5m 6s
680:	learn: 0.4651811	test: 0.4776899	best: 0.4776882 (676)	total: 10m 51s

759:	learn: 0.4641452	test: 0.4776414	best: 0.4776414 (758)	total: 12m 7s	remaining: 3m 49s
760:	learn: 0.4641354	test: 0.4776425	best: 0.4776414 (758)	total: 12m 8s	remaining: 3m 48s
761:	learn: 0.4641262	test: 0.4776426	best: 0.4776414 (758)	total: 12m 9s	remaining: 3m 47s
762:	learn: 0.4641152	test: 0.4776413	best: 0.4776413 (762)	total: 12m 10s	remaining: 3m 47s
763:	learn: 0.4641043	test: 0.4776421	best: 0.4776413 (762)	total: 12m 11s	remaining: 3m 46s
764:	learn: 0.4640951	test: 0.4776421	best: 0.4776413 (762)	total: 12m 12s	remaining: 3m 45s
765:	learn: 0.4640859	test: 0.4776422	best: 0.4776413 (762)	total: 12m 13s	remaining: 3m 44s
766:	learn: 0.4640760	test: 0.4776421	best: 0.4776413 (762)	total: 12m 14s	remaining: 3m 43s
767:	learn: 0.4640636	test: 0.4776467	best: 0.4776413 (762)	total: 12m 15s	remaining: 3m 42s
768:	learn: 0.4640546	test: 0.4776467	best: 0.4776413 (762)	total: 12m 17s	remaining: 3m 41s
769:	learn: 0.4640443	test: 0.4776492	best: 0.4776413 (762)	total: 12m 18

848:	learn: 0.4630825	test: 0.4776289	best: 0.4776148 (845)	total: 13m 36s	remaining: 2m 25s
849:	learn: 0.4630392	test: 0.4776182	best: 0.4776148 (845)	total: 13m 37s	remaining: 2m 24s
850:	learn: 0.4630292	test: 0.4776182	best: 0.4776148 (845)	total: 13m 38s	remaining: 2m 23s
851:	learn: 0.4630182	test: 0.4776189	best: 0.4776148 (845)	total: 13m 39s	remaining: 2m 22s
852:	learn: 0.4630076	test: 0.4776195	best: 0.4776148 (845)	total: 13m 40s	remaining: 2m 21s
853:	learn: 0.4629987	test: 0.4776195	best: 0.4776148 (845)	total: 13m 41s	remaining: 2m 20s
854:	learn: 0.4629879	test: 0.4776204	best: 0.4776148 (845)	total: 13m 42s	remaining: 2m 19s
855:	learn: 0.4629770	test: 0.4776206	best: 0.4776148 (845)	total: 13m 43s	remaining: 2m 18s
856:	learn: 0.4629681	test: 0.4776211	best: 0.4776148 (845)	total: 13m 44s	remaining: 2m 17s
857:	learn: 0.4629584	test: 0.4776226	best: 0.4776148 (845)	total: 13m 45s	remaining: 2m 16s
858:	learn: 0.4629484	test: 0.4776224	best: 0.4776148 (845)	total: 13m

937:	learn: 0.4621288	test: 0.4775797	best: 0.4775708 (902)	total: 15m 2s	remaining: 59.6s
938:	learn: 0.4621199	test: 0.4775797	best: 0.4775708 (902)	total: 15m 3s	remaining: 58.7s
939:	learn: 0.4620944	test: 0.4775774	best: 0.4775708 (902)	total: 15m 4s	remaining: 57.7s
940:	learn: 0.4620855	test: 0.4775775	best: 0.4775708 (902)	total: 15m 5s	remaining: 56.8s
941:	learn: 0.4620766	test: 0.4775770	best: 0.4775708 (902)	total: 15m 6s	remaining: 55.8s
942:	learn: 0.4620677	test: 0.4775770	best: 0.4775708 (902)	total: 15m 7s	remaining: 54.8s
943:	learn: 0.4620582	test: 0.4775774	best: 0.4775708 (902)	total: 15m 8s	remaining: 53.9s
944:	learn: 0.4620489	test: 0.4775780	best: 0.4775708 (902)	total: 15m 9s	remaining: 52.9s
945:	learn: 0.4620400	test: 0.4775782	best: 0.4775708 (902)	total: 15m 10s	remaining: 51.9s
946:	learn: 0.4620312	test: 0.4775775	best: 0.4775708 (902)	total: 15m 11s	remaining: 51s
947:	learn: 0.4620195	test: 0.4775777	best: 0.4775708 (902)	total: 15m 11s	remaining: 50s


In [34]:
import pickle
model = pickle.load(open('./model-CatBoostClassifier-0.sav','rb'))

In [38]:
import random
test_data['Ticket'] = test_data['Ticket'].fillna(random.choice(test_data['Ticket']))
test_data['Age'] = test_data['Age'].fillna(test_data['Age'].mean())
test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].mean())
test_data['Cabin'] = test_data['Cabin'].fillna(random.choice(test_data['Cabin']))
test_data['Embarked'] = test_data['Embarked'].fillna(random.choice(test_data['Embarked']))
test_data['Cabin'] = test_data['Cabin'].fillna(random.choice(test_data['Cabin']))

In [39]:
test_data.isna().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [40]:
test_data.to_csv('./test.csv',index=False)

In [42]:
new_embarked_col = []
for a in test_data['Embarked']:
    new_embarked_col.append(embarked_converter[a])
new_sex_col = []
for a in test_data['Sex']:
    new_sex_col.append(sex_converter[a])
test_data['Embarked'] = new_embarked_col
test_data['Sex'] = new_sex_col

In [43]:
preds = model.predict(ct.transform(test_data))
preds = np.round(preds)

In [44]:
submission = pd.DataFrame({'PassengerId':test_data['PassengerId'],'Survived':preds})

In [45]:
submission = submission.astype(int)

In [46]:
submission.dtypes

PassengerId    int64
Survived       int64
dtype: object

In [47]:
submission.to_csv('./2-submission.csv',index=False)

## Done